# Read Me

This project uses linkedin to send automatic messages in order to find a job faster

### Goals of the project
- Finding recruiters and people working in the industry i want and send them appropriate messages (adding the possibility to schedule a meeting with me on my personnal calendar ?
- Apply to the positions that fits my criteria (only working for simplified job apply imo)


In [31]:
from linkedin_api import Linkedin
from linkedin_api.utils.helpers import get_id_from_urn

In [12]:
# Authenticate using any Linkedin account credentials

test_api = Linkedin('tanguy.bellec@dauphine.eu', 'jrmmpt')
real_api = Linkedin('tangbellec@gmail.com', 'jrmmpt')


# GET a profile
#profile = api.get_profile('billy-g')

# GET a profiles contact info
#contact_info = api.get_profile_contact_info('billy-g')

# GET 1st degree connections of a given profile    USE URN ID
#connections = api.get_profile_connections('1234asc12304')


In [ ]:
def get_urn(profile):
    profile

In [4]:
profile = api.get_profile('tanguy-bellec-646992205')

In [5]:
profile

{'summary': 'Looking for a position as a Quant / Data Scientist for a data driven company where I can use my knowledge in Quantitative Finance / Statistics / Machine Learning to develop or to help developing investment strategy.\n- 1 year of experience as Data Scientist where I used ML to optimize insurance brokers portfolios\n- Strong knowledge in Mathematics for Finance (Itô stochastic calculus, SDE, PDE...)\n- Strong knowledge in most current ML techniques (supervised / unsupervised / reinforcement learning) \n- Programming language : Python\n- Librairies : Scikit-learn, Numpy, Pandas, PyTorch, Tensorflow\n- Environment : Hadoop (Spark)\n- Eager to learn\n- Web3 enthusiast\n- Chess lover',
 'industryName': 'Insurance',
 'lastName': 'Bellec',
 'locationName': 'France',
 'student': False,
 'geoCountryName': 'France',
 'geoCountryUrn': 'urn:li:fs_geo:105015875',
 'geoLocationBackfilled': False,
 'elt': False,
 'industryUrn': 'urn:li:fs_industry:42',
 'firstName': 'Tanguy',
 'entityUrn'

In [20]:
profile['entityUrn']

'urn:li:fs_profile:ACoAADRK5hYBGMJvZMHyTclwGIHUeJVuGK-eN_A'

In [13]:
contact_info = real_api.get_profile_contact_info('tanguy-bellec-646992205')

In [15]:
contact_info

{'email_address': 'tangbellec@gmail.com',
 'websites': [],
 'twitter': [],
 'birthdate': None,
 'ims': None,
 'phone_numbers': []}

In [17]:
connections = real_api.get_profile_connections('ACoAADRK5hYBGMJvZMHyTclwGIHUeJVuGK-eN_A')

In [18]:
connections

[{'urn_id': 'ACoAADN3mYwBhADFyxDujFBvdZjUCBSmP2ISAOs',
  'distance': 'DISTANCE_1',
  'public_id': 'hugo-jeanne-rose-9a4214201',
  'tracking_id': '863476108',
  'jobtitle': 'Alternant Data-analyst chez Generali / Master 2 Eco-stats à la Toulouse School of Economics',
  'location': 'Toulouse, Occitanie, France',
  'name': 'Hugo Jeanne-Rose'},
 {'urn_id': 'ACoAACUdCtQBnmdv6SnLJSlGk21Gx0SK1_xm43o',
  'distance': 'DISTANCE_1',
  'public_id': 'victor-pion-9b063b154',
  'tracking_id': '622660308',
  'jobtitle': 'Data Consultant chez Amaris Consulting',
  'location': 'France',
  'name': 'Victor Pion'},
 {'urn_id': 'ACoAAC003OwBByt01pOEQOqY5m3h1WEaPERqPYU',
  'distance': 'DISTANCE_1',
  'public_id': 'abdelkarim-eljandoubi',
  'tracking_id': '758439148',
  'jobtitle': 'Data scientist | ENSTA Paris - MVA ENS',
  'location': 'Massy, Île-de-France, France',
  'name': 'Abdelkarim Eljandoubi'},
 {'urn_id': 'ACoAABvzp1gBjBbwJbNQfcB0Sg6PP4rQPl2-Iok',
  'distance': 'DISTANCE_1',
  'public_id': 'nizar-ou

# First part : finding recruiters and sending them invite with personnal messages

- First dichotomy, from their summary we try to define if they are in the quant finance or in the tech sector to send the more accurate message

- Second one, if they are based in France we send our message in French

- To get better results we could developp a rating on how much we would be interested in each profile

In [162]:
def is_profile_data_or_finance(profile, data_keywords = ['tech','data','engineer', 'analyst', 'analytics'] , quant_finance_keywords = ['quantitative', 'finance', 'fintech', 'financial', 'hft', 'hedge', 'asset']):
    
    count_data = 0
    count_quant = 0
    
    # Some recruiters don't have a summary, in this case we take the title
    try:
        summary = profile['summary'].lower()

    except KeyError:
        summary = profile['headline'].lower()
        
    summary = summary.split(" ")
    
    for word in summary:
        
        if(word in data_keywords):
            count_data += 1
            
        if(word in quant_finance_keywords):
            count_quant += 1
            
    if(count_quant > count_data):
        return 'finance'
    
    if(count_data > 0):
        return 'data'
    
    return 'NA'


def is_profile_fetched_french(profile, france_keywords = ['france', 'paris', 'île-de-france']):
    
    try:
        location = profile['location'].lower()
        
    except KeyError:
        return False
    
    for keyword in france_keywords:
        if(keyword in location):
            return True
    
    return False


def get_first_name_fetched_profile(profile):
    
    try:
        first_name = profile['name'].split(" ")[0]
    
    except:
        return False
    return first_name



def add_personnalized_connection(current_recruiter, api = test_api, message_quant = message_quant,
                                 message_data = message_data, end_of_message = end_of_message):

    message = ""

    first_name = get_first_name_fetched_profile(current_recruiter)
    is_french = is_profile_fetched_french(current_recruiter)
    current_profile = test_api.get_profile(current_recruiter['public_id'])
    sector = is_profile_data_or_finance(current_profile)


    if(sector == 'finance'):
        message = "Hello " + first_name + "\n" + message_quant + "\n" + end_of_message

    elif(sector == 'data'):
        message = "Hello " + first_name + "\n" + message_data + "\n" + end_of_message

    else:  
        return "na_recruiter"

    connection = test_api.add_connection(current_recruiter['public_id'], message)
    
    return [sector, connection]

In [117]:
message_data = "I'm a graduate from an MSc in Statistcs from Université PSL in Paris, " \
    + "after a year of experience as a Data Scientist, I am looking for" \
    + "a new opportunity in this sector."   

message_quant = "I'm a graduate from an MSc in Statistcs and Quantitative Finance from Université " \
    + "PSL in Paris, after a year of experience as a Data Scientist, I am now looking to" \
    + "work as a Quant."  

end_of_message = "Feel free to contact me if you see a possible fit !"


data_recruiter = 0
quant_recruiter = 0
na_recruiter = 0

connections = []



In [ ]:
search_quant = real_api.search_people(keywords = 'quantitative finance', keyword_title = 'recruiter')

In [ ]:
for recruiter in search_quant:
    
    current_connection = test_add_personnalized_connection(recruiter, test_api)
    connections.append(current_connection)
    
    if(current_connection[0] == 'finance'):
        quant_recruiter += 1
    if(current_connection[0] == 'data'):
        data_recruiter += 1
    else:
        na_recruiter += 1

In [161]:
quant_recruiter

0

In [146]:
data_recruiter = 0
quant_recruiter = 0
na_recruiter = 0

current_recruiter = search[1]



In [144]:
na_recruiter

0

In [141]:
message_data

"I'm a graduate from an MSc in Statistcs from Université PSL in Paris, after a year of experience as a Data Scientist for the insurance company Generali, I am looking fora new opportunity in this sector."

In [40]:
search = real_api.search_people(keywords = 'data', keyword_title = 'recruiter')

In [41]:
search

[{'urn_id': 'ACoAAC3hoMABc0QYhzf2kzfdbL-Xq9ecIdWoEFw',
  'distance': 'DISTANCE_2',
  'public_id': 'agneseducceschi',
  'tracking_id': '769761472',
  'jobtitle': 'Recruiter & Executive Assistant at Boston Consulting Group (BCG)',
  'location': 'Paris, Île-de-France, France',
  'name': 'Agnese Ducceschi'},
 {'urn_id': 'ACoAABrwlW8BVdW6pfWL1cMLSzJaON5lU4MDjBc',
  'distance': 'DISTANCE_3',
  'public_id': 'gwendolinec',
  'tracking_id': '451974511',
  'jobtitle': 'Tech Recruiter & Business Developper',
  'location': 'Aix-les-Bains, Auvergne-Rhône-Alpes, France',
  'name': 'Gwendoline Chardon'},
 {'urn_id': 'ACoAACYwHO0BmrDsWnzgq5RsQ3Xfyfeqo0O2U8w',
  'distance': 'DISTANCE_2',
  'public_id': 'rizzoevelin',
  'tracking_id': '640687341',
  'jobtitle': 'Global Campus Recruiting @ Eurofins Scientific 🔬🧬🎓',
  'location': 'Brussels Metropolitan Area',
  'name': 'Evelin Rizzo (she / her)'},
 {'urn_id': 'ACoAABngCBYBEENyxhOOfh5srOAVgLFaL15CvLA',
  'distance': 'DISTANCE_2',
  'public_id': 'merwan-el-

In [80]:
new_search = real_api.search_people(keywords = 'quantitative finance', keyword_title = 'recruiter')

In [81]:
new_search

[{'urn_id': 'ACoAAB2GeYcBpawNZAg2bVIgMc_AQu7asPcmmkc',
  'distance': 'DISTANCE_3',
  'public_id': 'maggie-tamoney',
  'tracking_id': '495352199',
  'jobtitle': 'Hiring Machine Learning/AI Engineers at Facebook & Instagram',
  'location': 'Greater Boston',
  'name': 'Maggie Tamoney'},
 {'urn_id': 'ACoAAC5n-qsBW3UrI0fzn07ycT3a1sLiGAwwsPw',
  'distance': 'DISTANCE_3',
  'public_id': 'sajsanthirakumar',
  'tracking_id': '778566315',
  'jobtitle': 'Recruiter - Quantitative Trading and Technology',
  'location': 'London, England, United Kingdom',
  'name': 'Sajeevan Santhirakumar'},
 {'urn_id': 'ACoAAAFNJY0BrbnNpf80B2mwiVssLXnSpfDiZn8',
  'distance': 'DISTANCE_2',
  'public_id': 'emeriqueopou',
  'tracking_id': '21833101',
  'jobtitle': 'Experte en recrutement actuariat, finance quantitative, risque, conformité - Fondatrice & CEO - Emérique & Partners',
  'location': 'Paris, Île-de-France, France',
  'name': 'Emérique Opou'},
 {'urn_id': 'ACoAAA8NLakB1iRNZlDsshrM97-Vw--vLlt3Irk',
  'distance

In [93]:
test_api.get_profile('freddy-green-68a5bb10a')

{'summary': "For the last 7 years I've been connecting technologists around the world with some of the most exciting organisations in the Financial Services industry. \n\nNow, I'm working at Durlston Partners, a specialist agency that pride ourselves on partnering (often exclusively) with boutique, high-growth Hedge Funds, Asset Managers and Digital Asset companies. Our client-base is made up of well-established start-up's and future-multi-billion dollar hedge funds, giving our candidate network access to unusual opportunities with exceptional growth potential on the buy-side",
 'industryName': 'Staffing & Recruiting',
 'lastName': 'Green',
 'locationName': 'United Kingdom',
 'student': False,
 'geoCountryName': 'United Kingdom',
 'geoCountryUrn': 'urn:li:fs_geo:101165590',
 'geoLocationBackfilled': False,
 'elt': False,
 'industryUrn': 'urn:li:fs_industry:104',
 'firstName': 'Freddy',
 'entityUrn': 'urn:li:fs_profile:ACoAABt_ZQoBNlocxfCznX7JnNnLch3-KS2xSKg',
 'geoLocation': {'geoUrn':

In [79]:
fetched_profile

{'summary': 'An ambitious data driven professional with strong accounts management and leadership skills. Excellent interpersonal, analytical problem-solving and organizational skills. Ability to work independently or as part of a team; multi-task; balance multiple priorities; and meet deadlines in a fast-paced global corporate environment. Acquired firsthand working knowledge and overall view of departmental operations, including budget, personnel, and programming.\n\nOutside of my career I am the proud owner of Teka’s Timeless Bouquets, a handmade flower bouquet company. It is my pleasure to help customers create timeless memories for their special occasions. ',
 'industryName': 'Accounting',
 'lastName': 'Scafe',
 'locationName': 'United States',
 'student': False,
 'geoCountryName': 'United States',
 'geoCountryUrn': 'urn:li:fs_geo:103644278',
 'geoLocationBackfilled': False,
 'elt': False,
 'industryUrn': 'urn:li:fs_industry:47',
 'firstName': 'Nateka',
 'entityUrn': 'urn:li:fs_pr

In [59]:
test_api.get_profile('evan-robertson-10348869')

{'summary': 'Hoxton Circle is a boutique search and recruiting firm specialising in permanent, direct hire and interim staffing and recruiting services for Senior Appointments and Mid-to-Senior Level Accounting and Finance experts, Finance Systems and Projects in NYC. Established in 2016, we’ve grown a reputation within the industry for delivering world-class service and developing exceptional partnerships with our clients and candidates. \n\nWith over 75 years of combined recruitment experience, Hoxton Circle can provide a bespoke service to their clients and candidates with unrivalled recruitment expertise and specific industry knowledge. In a competitive market, Hoxton Circle has developed relationships with both public and private companies, delivering quality where others have not. \n\nWe offer impartial advice to both active and passive job seekers and ensure the recruitment process is one that suits their needs. With excellent attention to detail and a strong network of contacts